In [ ]:
from os import path

# Third-party
from astropy.table import Table
import astropy.coordinates as coord
import astropy.units as u
from astropy.constants import G, c
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.stats import scoreatpercentile
from scipy.signal import argrelmin

from comoving_rv.log import logger
from comoving_rv.db import Session, Base, db_connect
from comoving_rv.db.model import (Run, Observation, TGASSource, SimbadInfo, PriorRV,
                                  SpectralLineInfo, SpectralLineMeasurement, RVMeasurement)

import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

# import matplotlib as mpl
# mpl.rc('text', usetex=True)
# mpl.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}"]

In [ ]:
gc_frame = coord.Galactocentric(z_sun=0*u.pc)

In [ ]:
base_path = '/Volumes/ProjectData/gaia-comoving-followup/'
db_path = path.join(base_path, 'db.sqlite')
engine = db_connect(db_path)
session = Session()

In [ ]:
tbl = Table.read('group_llr_dv_tbl.ecsv', format='ascii.ecsv')

In [ ]:
group_ids = tbl['group_id'][tbl['R_RV'] > tbl['R_mu']]

Ugh, so bad - copied from other notebook. This is for cases where a group has >2 observations

In [ ]:
multi_obs = {
    942: [2,3],
    1497: [0,1],
    1617: [0,1],
    1958: [0,1],
    2308: [2,3],
    3196: [0,1],
    3230: [0,1],
    3376: [2,3],
    3455: [0,1],
    4399: [0,1],
    1397: [1,2],
    1942: [0,1],
    1992: [0,1],
    2499: [1,2],
    2633: [0,1],
    3245: [0,1],
    3287: [0,2],
    3580: [0,2],
    4373: [1,2],
}

## Integrate orbits, look at orbital properties

In [ ]:
mw_pot = gp.CCompositePotential()
mw_pot['spheroid'] = gp.HernquistPotential(m=5E9, c=0.7, units=galactic)
mw_pot['halo'] = gp.NFWPotential(m=5E11*u.Msun, r_s=16., units=galactic)
mw_pot['disk1'] = gp.MiyamotoNagaiPotential(m=6.5E10*u.Msun, a=3, b=0.273,
                                            units=galactic)

In [ ]:
def get_Jz(orbit):
    # Estimate Jz using epicycle approx. (pg. 232 in B&T)
    t_cross = orbit.t[argrelmin(np.abs(orbit.pos.xyz[2]))]
    Tz = np.mean(t_cross[1:] - t_cross[:-1])
    nu = 2*np.pi / Tz
    Ez = 0.5 * (orbit.vel.d_xyz[2,0]**2 + nu**2 * orbit.pos.xyz[2,0]**2)
    return (Ez / nu).to(u.kpc*u.km/u.s)

In [ ]:
sun_w0 = gd.PhaseSpacePosition(pos=[-8.3,0,0]*u.kpc, vel=gc_frame.galcen_v_sun.d_xyz)
sun_orbit = mw_pot.integrate_orbit(sun_w0, dt=0.1, n_steps=80000)
sun_max_z = np.abs(sun_orbit.z).max()
sun_Jz = get_Jz(sun_orbit)

In [ ]:
Jz = []
max_z = []
for gid in group_ids:
    observations = session.query(Observation).filter(Observation.group_id == gid).all()
    
    if gid in multi_obs:
        obs1 = observations[multi_obs[gid][0]]
        obs2 = observations[multi_obs[gid][1]]
    else:
        obs1, obs2 = observations

    icrs1 = obs1.icrs(with_rv=True, lutz_kelker=True)
    icrs2 = obs2.icrs(with_rv=True, lutz_kelker=True)
    
    c1 = icrs1.transform_to(gc_frame)
    c2 = icrs2.transform_to(gc_frame)
    
    x0 = np.mean([c1.cartesian.xyz, c2.cartesian.xyz], axis=0) * u.pc
    v0 = np.mean([c1.cartesian.differentials['s'].d_xyz,
                  c2.cartesian.differentials['s'].d_xyz], axis=0) * u.km/u.s
    
    w0 = gd.PhaseSpacePosition(pos=x0, vel=v0)
    orbit = mw_pot.integrate_orbit(w0, dt=0.1, n_steps=80000)
    
    Jz.append(get_Jz(orbit))
    max_z.append(np.abs(orbit.z).max())
    
max_z = u.Quantity(max_z)
Jz = u.Quantity(Jz)

In [ ]:
plt.figure(figsize=(5,5))
plt.hist(Jz.value, bins='auto')
plt.axvline(sun_Jz.value, color='#aaaaaa', linewidth=3, linestyle='--')
plt.xlabel('$J_z$ [{0}]'.format(Jz.unit.to_string('latex_inline')))
plt.text(sun_Jz.value+0.1, 35, r'$J_{z, \odot}$', fontsize=20, color='#666666')
plt.ylim(0, 40)

In [ ]:
plt.figure(figsize=(5,5))
plt.hist(max_z.to(u.pc).value, bins='auto')
plt.axvline(sun_max_z.to(u.pc).value, color='r')

## Angular momentum of pairs:

In [ ]:
all_L = []
for gid in group_ids:
    observations = session.query(Observation).filter(Observation.group_id == gid).all()
    
    if gid in multi_obs:
        obs1 = observations[multi_obs[gid][0]]
        obs2 = observations[multi_obs[gid][1]]
    else:
        obs1, obs2 = observations

    # Compute point-estimate difference in 3D velocity
    icrs1 = obs1.icrs(with_rv=True)
    icrs2 = obs2.icrs(with_rv=True)
    
    c1 = icrs1.transform_to(coord.Galactocentric)
    c2 = icrs2.transform_to(coord.Galactocentric)
    
    x1 = c1.cartesian.xyz
    x2 = c2.cartesian.xyz
    dx = x1 - x2
    
    v1 = c1.cartesian.differentials['s'].d_xyz
    v2 = c2.cartesian.differentials['s'].d_xyz
    dv = v1 - v2
    
    all_L.append(np.cross(dx, dv))
    
all_L = u.Quantity(all_L).T

In [ ]:
alignment = np.arccos(all_L[2] / np.linalg.norm(all_L, axis=0))

In [ ]:
plt.hist(np.arccos(2*np.random.uniform(size=10000) - 1) * 180/np.pi, 
         normed=True, color='#888888', alpha=0.5)
plt.hist(alignment.to(u.degree).value, bins=np.linspace(0, 180, 15), normed=True,
         color='k', alpha=0.75);